In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --------------------------- ------------ 1.0/1.5 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00


In [25]:
pip install pyodbc

   ---------------------------------------- 0.0/68.9 kB ? eta -:--:--
   ----------------------------------- ---- 61.4/68.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 68.9/68.9 kB 945.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
pip install --upgrade nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\agraw\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=GARIMA\SQLSERVER;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )

    conn = pyodbc.connect(conn_str)
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM fact_customer_reviews"
    df = pd.read_sql(query,conn)
    conn.close()
    return df

In [ ]:
customer_reviews_df = fetch_data_from_sql()
sia = SentimentIntensityAnalyzer()

In [ ]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [ ]:
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating>=4:
            return 'Positive'
        elif rating==3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score< -0.05:
        if rating<=2:
            return 'Neagtive'
        elif rating==3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating>=4:
            return 'Positive'
        elif rating<=2:
            return 'Negative'
        else:
            return 'Neutral'

In [ ]:
def sentiment_bucket(score):
    if score>=0.5:
        return '0.5 to 1.0'
    elif 0.0<=score<0.5:
        return '0.0 to 0.49'
    elif -0.5<=score<0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [ ]:
customer_reviews_df['SentimentCategory'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']),axis=1
)

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

print(customer_reviews_df.head())

customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv',index=False)